In [1]:
import pandas as pd
from datetime import datetime, timedelta
import json
import requests
import dotenv
import os

dotenv.load_dotenv(".env")

True

In [2]:
df_temp = pd.read_csv('data/weather/temperature.csv')
df_temp

,date,temperature
0,2022-01-01 00:00:45,21.035860
1,2022-01-01 00:01:50,21.020624
2,2022-01-01 00:02:54,21.000704
3,2022-01-01 00:03:59,20.974922
4,2022-01-01 00:05:04,20.947382
...,...,...
384999,2022-12-19 20:26:38,22.543673
385000,2022-12-19 20:27:43,22.540352
385001,2022-12-19 20:28:48,22.541523
385002,2022-12-19 20:29:54,22.536640


In [3]:
def date_time_getter(combined_date):
  date = combined_date.split(' ')[0]
  time = combined_date.split(' ')[1]
  return date, time

df_temp[['actual_date', 'actual_time']] = df_temp['date'].apply(lambda x: pd.Series(date_time_getter(x)))
# df_temp['actual_date'] = pd.to_datetime(df_temp['actual_date'])

df_temp

: 

In [4]:
df = df_temp.copy()

df.drop(columns='date',inplace=True)
df = df[['actual_date','actual_time','temperature']] 
df.rename(columns = {'actual_date':'date','actual_time':'time'}, inplace = True)
df.head()

,date,time,temperature
0,2022-01-01,00:00:45,21.035860
1,2022-01-01,00:01:50,21.020624
2,2022-01-01,00:02:54,21.000704
3,2022-01-01,00:03:59,20.974922
4,2022-01-01,00:05:04,20.947382


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385004 entries, 0 to 385003
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   date         385004 non-null  object 
 1   time         385004 non-null  object 
 2   temperature  385004 non-null  float64
dtypes: float64(1), object(2)
memory usage: 8.8+ MB


In [6]:
def aggregate_temperatures_per_hour(df):
    # Convert 'time' column to datetime object
    df['time'] = pd.to_datetime(df['time'],format="%H:%M:%S")
    
    # Extract hour component from 'time' column
    df['hour'] = df['time'].dt.hour
    
    # Group by 'date' and 'hour' and calculate mean temperature
    aggregated_df = df.groupby(['date', 'hour'])['temperature'].mean().reset_index()
    
    return aggregated_df
  
df_aggregate = aggregate_temperatures_per_hour(df)
df_aggregate

,date,hour,temperature
0,2022-01-01,0,20.640966
1,2022-01-01,1,20.200706
2,2022-01-01,2,19.873540
3,2022-01-01,3,19.522044
4,2022-01-01,4,20.097169
...,...,...,...
7515,2022-12-19,16,23.728903
7516,2022-12-19,17,23.288150
7517,2022-12-19,18,22.806882
7518,2022-12-19,19,22.649602


In [3]:
cp = pd.read_json('data/carparks_original.json')
cp.head()

,facility_id,CarParkName
0,1,Tallawong Station Car Park (historical only)
1,10,Warriewood Car Park
2,11,Narrabeen Car Park
3,12,Mona Vale Car Park
4,13,Dee Why Car Park


In [4]:

coords_df = pd.read_json('data/coords.json')
coords_df.head()

,facility_id,longitude,latitude
0,6,-33.756009,151.154528
1,7,-34.672518,150.854695
2,8,-33.423883,151.341711
3,9,-33.952460,151.014838
4,10,-33.697777,151.300667


In [5]:
final_df = cp.copy()

final_df = final_df.merge(coords_df,on='facility_id',how='left')

final_df.sort_values(by='facility_id')

,facility_id,CarParkName,longitude,latitude
0,1,Tallawong Station Car Park (historical only),NaN,NaN
11,2,Kellyville Station Car Park (historical only),NaN,NaN
22,3,Bella Vista Station Car Park (historical only),NaN,NaN
27,4,Hills Showground Station Car Park (historical ...,NaN,NaN
33,5,Cherrybrook Station Car Park (historical only),NaN,NaN
34,6,Gordon Henry St North Car Park,-33.756009,151.154528
35,7,Kiama Car Park,-34.672518,150.854695
36,8,Gosford Car Park,-33.423883,151.341711
37,9,Revesby Car Park,-33.952460,151.014838
1,10,Warriewood Car Park,-33.697777,151.300667


In [7]:
df_final = pd.read_parquet('data/carpark_history_3_months_combined.parquet')
df_final.head(10)

,tsn,time,spots,zones,ParkID,occupancy,MessageDate,facility_id,facility_name,tfnsw_facility_id
0,2101131,749399962,244,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-10-01T00:39:22,10,Warriewood Car Park,2101131TPR001
1,2101131,749422317,244,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-10-01T07:51:57,10,Warriewood Car Park,2101131TPR001
2,2101131,749431023,244,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-10-01T10:17:03,10,Warriewood Car Park,2101131TPR001
3,2101131,749431177,244,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-10-01T10:19:37,10,Warriewood Car Park,2101131TPR001
4,2101131,749431421,244,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-10-01T10:23:41,10,Warriewood Car Park,2101131TPR001
5,2101131,749431470,244,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-10-01T10:24:30,10,Warriewood Car Park,2101131TPR001
6,2101131,749432420,244,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-10-01T10:40:20,10,Warriewood Car Park,2101131TPR001
7,2101131,749435227,244,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-10-01T11:27:07,10,Warriewood Car Park,2101131TPR001
8,2101131,749444612,244,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-10-01T14:03:32,10,Warriewood Car Park,2101131TPR001
9,2101131,749445426,244,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-10-01T14:17:06,10,Warriewood Car Park,2101131TPR001


In [8]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1691049 entries, 0 to 1691048
Data columns (total 10 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   tsn                object
 1   time               object
 2   spots              object
 3   zones              object
 4   ParkID             object
 5   occupancy          object
 6   MessageDate        object
 7   facility_id        object
 8   facility_name      object
 9   tfnsw_facility_id  object
dtypes: object(10)
memory usage: 129.0+ MB
